In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
stazioni = pd.read_csv('stazioni mancanti.csv', header=None)

In [ ]:
regioni = []

for codice in stazioni[1]:
    regioni.append(requests.get("http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno/regione/" + codice).text)

stazioni['regioni'] = regioni

In [ ]:
lat = []
lon = []

for riga in stazioni.itertuples():
    if riga[3]:
        testo = requests.get("http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno/dettaglioStazione/" + riga[2] + '/' + riga[3]).text
        testo = testo.split("\"lat\":")
        testo = testo[1].split(",\"lon\":")
        lat.append(testo[0])
        testo = testo[1].split(",\"latMappaCitta\"")
        lon.append(testo[0])
    else:
        lat.append('')
        lon.append('')

stazioni['lat'] = lat
stazioni['lon'] = lon

In [ ]:
ids = []

for riga in stazioni.itertuples():
    if riga[4] and riga[5]:
        page = requests.get("https://wikidata.reconci.link/it/api?queries=%7B%22q0%22%3A%7B%22query%22%3A%22stazione+"+
                            riga[1]+"%22%2C%22properties%22%3A%5B%7B%22pid%22%3A%22P625%22%2C%22v%22%3A%22"+riga[4]+
                            "%2C"+riga[5]+"%22%7D%5D%7D%7D")
        if page.text == "{\"q0\":{\"result\":[]}}":
            print(riga[1], "nessun match")
            ids.append('')
            continue
        id = page.text.split(",\"id\":\"")
        id = id[1].split("\"")[0]
        ids.append(id)
        print(riga[1], id)
    else:
        page = requests.get("https://wikidata.reconci.link/it/api?queries=%7B%22q0%22%3A%7B%22query%22%3A%22stazione+"+riga[1]+"%22%7D%7D")
        if page.text == "{\"q0\":{\"result\":[]}}":
            print(riga[1], "nessun match")
            ids.append('')
            continue
        id = page.text.split(",\"id\":\"")
        id = id[1].split("\"")[0]
        ids.append(id)
        print(riga[1], id)

stazioni['ids'] = ids

In [ ]:
wiki_urls = []

for riga in stazioni.itertuples():
    if riga[6]:
        page = requests.get("https://www.wikidata.org/wiki/" + riga[6])
        soup = BeautifulSoup(page.content, 'html.parser')
        pagina = soup.find("span",class_="wikibase-sitelinkview-page",lang="it")
        if pagina:
            url = pagina.find("a")
            wiki_urls.append(url['href'])
        else:
            wiki_urls.append('')
    else:
        wiki_urls.append('')

stazioni['wiki_urls'] = wiki_urls

In [ ]:
pagine_ok = []
errori = 0
cose_sconosciute = [[],[]]

for riga in stazioni.itertuples():
    if riga[7]:

        page = requests.get(riga[7])
        a = page.status_code

        # scrape webpage
        if(a==200):
            soup = BeautifulSoup(page.content, 'html.parser')
            if soup.find("th", text="Binari"):
                binari = soup.find("th", text="Binari").find_next_sibling("td").text
                pagine_ok.append(binari)
            else:
                errori+=1
                pagine_ok.append('')

        elif(a==404):
            errori+=1
            pagine_ok.append('')

        else:
            cose_sconosciute[0].append(riga[7])
            cose_sconosciute[1].append(a)
            pagine_ok.append('')

        print(riga[1], binari)
    else:
        pagine_ok.append('')

print(len(pagine_ok), errori)


stazioni['binari'] = pagine_ok

In [ ]:
stazioni_ok = stazioni[stazioni['binari'].notnull()]
stazioni_ok['binari'] = stazioni_ok['binari'].str.replace('\n', '')
stazioni_ok['binari_ok'] = stazioni_ok['binari'].str.replace(r'([0-9]*).*', r'\1', regex=True)
stazioni_ok = stazioni_ok[stazioni_ok['binari_ok'] != '']
stazioni_ok['binari_ok'].apply(int)
stazioni_ok.rename(columns={"0":"nome_stazione", "1":"id_staz"}, inplace=True)
stazioni_ok.set_index("id_staz", inplace=True, drop=True)
stazioni_ok.drop('Unnamed: 0', axis=1, inplace=True)
stazioni_ok.to_csv('binari wiki.csv')